# Aula Prática (P01): Criação de parâmetros e Minimização de energia de uma Haleto Methyltransferase (PDB 3LCC)

**Objetivos:**
- Estudar o artigo de Schmidberger et. al. (DOI:10.1002/anie.201000119) para entender os parâmetros cinéticos para Cl⁻, Br⁻ e SCN⁻  
- Preparar o complexo proteína–SAM–íon no AMBER  
- Executar minimização MM e QM/MM  

**Estrutura utilizada:**  
PDB **3LCC**, estrutura de halide methyltransferase de *Arabidopsis thaliana*

# 1) Preparação do Complexo Michaelis – Halide Methyltransferase (3LCC)


## 1.1 Download do PDB 3LCC

Baixe o arquivo do PDB diretamente do repositório da RCSB.
Você pode acessar https://www.rcsb.org/structure/3LCC ou, no terminal, digitar: wget -O 3LCC.pdb https://files.rcsb.org/download/3LCC.pdb
Utilize a célula a seguir, se quiser executar diretamente do jupyter-notebook



In [1]:
%%bash
echo "Baixando o PDB 3LCC da RCSB..."
wget -O 3LCC.pdb https://files.rcsb.org/download/3LCC.pdb
echo "Arquivo baixado: 3LCC.pdb"


Baixando o PDB 3LCC da RCSB...


--2025-05-15 14:26:48--  https://files.rcsb.org/download/3LCC.pdb
Resolvendo files.rcsb.org (files.rcsb.org)... 108.139.134.60, 108.139.134.86, 108.139.134.26, ...
108.139.134.60|:443... conectado.iles.rcsb.org)|
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: não especificada [application/octet-stream]
Salvando em: ‘3LCC.pdb’

..... .......... .......... ..........  392K
. .......... .......... .......... ..........  396K
 .......... .......... .......... .......... 4,08M
.......... .......... .......... .......... ..         5,44M=0,3s

2025-05-15 14:26:50 (703 KB/s) - ‘3LCC.pdb’ salvo [196830]



Arquivo baixado: 3LCC.pdb


---
## 1.2 Visualização do PDB no UCSF Chimera

Vamos abrir o PDB original no Chimera para inspeção e posterior edição.

> **Passo 1:** Abra o arquivo no Chimera pelo terminal:
>
> ```sh
> chimera 3LCC.pdb
> ```

---

> **Passo 2:**
> - Ao abrir o arquivo, navegue pela estrutura e localize o cofator SAM (que está na forma SAH), as moléculas de água que darão lugar aos íons e o sítio ativo da enzima.
> - Identifique os resíduos de aminoácidos que interagem com o SAM
> - Você deverá editar o arquivo PDB para simular o **complexo de Michaelis**, conforme ilustrado na **Figura 2** do artigo de referência.
---
> **Passo 3:**
> - Utilize os recursos de seleção e visualização do Chimera para editar (Tools / Structure Editing / Build Structure)
> - Adicione hidrogênios à estrutura (Tools / Structure Editing / AddH)
> - Após as edições, salve um novo arquivo PDB com o nome `3LCC_Michaelis.pdb`, que será utilizado nas próximas etapas da simulação.




---
## 1.3 Determinação do Estado de Protonação dos Resíduos

Antes de avançar para as etapas de simulação, é fundamental **verificar e ajustar o estado de protonação** dos resíduos de aminoácidos, pois isso pode impactar significativamente os resultados de energia e o mecanismo catalítico modelado. O estado de protonação afeta as cargas parciais dos resíduos, especialmente nos sítios ativos (como ácidos, bases e histidinas).

---

**Passo 1: Acesse o [servidor H++](https://biophysics.cs.vt.edu/H++/index.php):**  
   [http://newbiophysics.cs.vt.edu/](http://newbiophysics.cs.vt.edu/)
    Digite login e senha para continuar
    
**Passo 2: Faça o upload do arquivo PDB** editado, correspondente ao complexo de Michaelis (`3LCC_Michaelis.pdb`).

**Passo 3: Selecione o pH de interesse** (recomenda-se pH fisiológico, em torno de 7.0).
    No entanto, é essencial que verifique o artigo publicado para descobrir a que pH foram feitos os experimentos cinéticos.
    
**Passo 4: Envie para processamento** e aguarde o resultado e clique em 'View Results'

**Passo 5: Baixe o novo arquivo PDB** protonado, que será utilizado nas etapas seguintes da preparação do sistema:
     Other Useful Files
     AMBER compatible PDB (PQR) structure in predicted protonation state: 0.15_80_10_pH6_3LCC.genpdb.cpptraj.pdb

> ⚠️ **Dica:**  
> Revise cuidadosamente a protonação de resíduos-chave, como histidinas, cisteínas e ácidos/asparaginas próximos ao sítio catalítico. Caso necessário, faça ajustes manuais no Chimera.

---

## 1.5 Preparação dos Parâmetros para o Complexo Proteína-Ligante

Finalmente, vamos gerar todos os arquivos necessários para simulações no AMBER, começando pela criação de uma **biblioteca (.lib)** para o SAM e, em seguida, preparando o complexo proteína-ligante já solvatado.

---

**Passo 1**: Gerar o arquivo .lib para o Cofator SAM. 

> O arquivo .lib (ou .off) no AMBER é uma biblioteca que contém todas as informações químicas necessárias para descrever um novo resíduo ou molécula não padrão, como ligantes ou cofatores. Ele armazena a conectividade, tipos de átomos e cargas parciais, permitindo que o AMBER reconheça e utilize corretamente essa molécula durante a preparação e simulação do sistema. Esse arquivo é gerado após parametrização e é essencial para incluir moléculas especiais que não estão presentes nos force fields padrão do AMBER.

Abra o **tLeap** (no terminal linux, digite *tleap* e execute os comandos abaixo (ajuste os nomes dos arquivos conforme seu sistema):

```sh
tleap -f - <<EOF
source leaprc.gaff2             # Carrega o force field GAFF2 (ideal para SAM)
loadamberparams SAM.frcmod      # Parâmetros complementares do SAM, gerados anteriormente
SAM = loadmol2 SAM.mol2         # Carrega as coordenadas e cargas do SAM (.mol2)
saveoff SAM SAM.lib             # Salva biblioteca com os parâmetros do SAM
saveamberparm SAM SAM.parm7 SAM.inpcrd  # Arquivos topologia e coordenadas do SAM isolado
quit
EOF

**Passo 2**: Gerar o sistema completo: proteína, SAM, íon Cl⁻ e solvente. 

tleap -f - <<EOF
source leaprc.protein.ff14SB    # Force field para proteínas
source leaprc.gaff              # Force field para moléculas orgânicas (SAM)
source leaprc.water.tip3p       # Modelo de água TIP3P
loadamberparams SAM.frcmod      # Parâmetros do SAM
loadoff SAM.lib                 # Biblioteca do SAM criada anteriormente
com = loadpdb 3LCC_Michaelis_amber.pdb  # Carrega o PDB preparado

# Solvatar o sistema em uma caixa de 12 Å
solvateBox com TIP3PBOX 12.0

# Verificar a carga total do sistema (deve ser inteiro)
charge com

# Neutralizar o sistema adicionando íons Na+
addIons com Na+ 0

# Salvar arquivos finais de topologia, coordenadas e PDB do sistema solvatado
saveamberparm com 3lcc_CL.prmtop 3lcc_CL.inpcrd
savepdb com 3lcc_CL_box.pdb
quit
EOF


## 1.5 Análise da minimização AMBER

Vamos extrair e plotar a energia da minimização do arquivo `min4.cl.out`
